# QMUL Careers Team Python Script


In [10]:
# The first cell imports some of the software that the programme needs, if you click the small play button above, or press Shift and Enter, it will run the cell.

In [4]:
import pandas as pd
import openpyxl
import numpy as np
import os
import ipywidgets

In [9]:
def get_sheet():
    for file in os.listdir('.'):
        if file.endswith(".xlsx"):
            myfile = file
            print(f'{file} has been selected.')
        else:
            pass           
    try:
        return myfile
    except:
        print('No file has been uploaded')
    
grabbit = get_sheet()
df = pd.read_excel(grabbit)
for i in df.columns:
    df.rename(columns={i:i.lower()}, inplace=True)

criteria = [ 
'student number',
'year of study',
'gender',
'faculty',
'total',
'course',
'personality type']

In [ ]:
for_sorting = ['total']

# to add more items to sort by, you need to add them to this list here.

In [ ]:
# Any missing scores will be replaced with the average score of the dataset.

In [ ]:
df['total'].fillna(df['total'].mean(), inplace=True)

### Gender Distribution

In [ ]:
df['gender'].value_counts()

### Year of Study Distribution

In [ ]:
df['year of study'].value_counts()

In [ ]:
group_df = df[criteria]

In [ ]:
group_df = group_df.assign(groupno='')


In [ ]:
# The Sorting Algorithm

In [ ]:
def assign_group(df):
    df.sort_values(by=for_sorting, ascending=False, inplace=True)
    count = 1
    up = True
    for i,row in group_df.iterrows():

        if count <= round(len(df)/5,0) and up == True:
            df.at[i,'groupno'] = count
            count += 1

        elif count > round(len(df)/5,0) and up == True:
            df.at[i,'groupno'] = count-1
            count-=1
            up = False

        if count > 1 and up == False:
            df.at[i,'groupno'] = count
            count -= 1
            
        elif count == 1 and up == False:
            df.at[i,'groupno'] = count
            up = True
    
    return df

assigned = assign_group(group_df)

    

## Data Analysis

### Average Score by Group

In [ ]:
assigned.groupby(['groupno'])[['total']].mean().round(2).min()

### Year of Study per Group

In [ ]:
assigned.pivot_table(index=['groupno'], columns=['year of study'], aggfunc='size', fill_value=0)


### Gender per Group

In [ ]:
assigned.pivot_table(index=['groupno'], columns=['gender'], aggfunc='size', fill_value=0)


### Click the download button below for applications assigned to groups

In [ ]:
subset = assigned[['student number','groupno']]

final = pd.merge(df,subset, how='inner', on='student number')



In [ ]:
final.to_excel("grouped_student_applications.xlsx",sheet_name='Grouped Students') 

In [ ]:
import urllib.parse
def build_csv_data(dataframe):
    csv_data = dataframe.to_csv(index=False, encoding='utf-8')
    return csv_data

In [ ]:
from ipywidgets import Output, Button
from IPython.display import HTML, clear_output
from base64 import b64encode

download_output = Output()
display(download_output)
def trigger_download(filename, kind='text/csv'):
    # see https://developer.mozilla.org/en-US/docs/Web/HTTP/Basics_of_HTTP/Data_URIs for details
    text = build_csv_data(final)
    content_b64 = b64encode(text.encode()).decode()
    data_url = f'data:{kind};charset=utf-8;base64,{content_b64}'
    js_code = f"""
        var a = document.createElement('a');
        a.setAttribute('download', '{filename}');
        a.setAttribute('href', '{data_url}');
        a.click()
    """
    with download_output:
        clear_output()
        display(HTML(f'<script>{js_code}</script>'))

btn = Button(description='Download Groups')
def download_groups(e=None):
    trigger_download('applications_grouped.csv', kind='text/csv')
btn.on_click(download_groups)
display(btn)